# The Combinatorial Explosion
---

## Back to basics: forget your chemistry

With a whole periodic table of elements at our disposal, the number of possible combinations of elements that might form useful materials is truly mind-boggling. [This blog entry](http://hackingmaterials.com/2013/11/14/the-scale-of-materials-design) highlights this well with the following thought experiment:

1. You have at your disposal the first 50 elements of the periodic table
2. You also have a 10 x 10 x 10 grid 
3. You are allowed to arrange 30 of the elements at a time in some combination in the grid to make a 'compound'
4. How many different arrangements (different compounds) could you make?

<img src = "Images/atomsinbox.png">

The answer is about $10^{108}$, *over a googol of compounds!* 

This is a totally unimaginable number (there are an estimated $10^{80}$ atoms in the universe), so clearly this kind of blind exploration is not feasable. Fortunately, centuries of chemical research has provided us with many rules that could be applied to assess the feasability of combinations.

First off, we know that all the compounds we have so far adopt one of a finite set of crystal structures. We also know that compounds rarely exhibit > 5 different elements. This already simplifies the problem substantially but we're not out of the woods yet. We are still left with many trillions of possibilities and the choice of how to narrow these down will depend heavily on what the end goal is in any specific case. 

This practical explores how materials design can be approached by using the simplest of rules in order to narrow down the combinations to those that might be considered legitimate. It will hopefully demonstrate the scale of the problem, even after the rules are applied. 

 ***"During the past century, science has developed a limited capability to design materials, but we are still too dependent on serendipity"*** - [Eberhart and Clougherty, Looking for design in materials design (2004)](http://www.nature.com/nmat/journal/v3/n10/abs/nmat1229.html)

## Counting combinations: remember your chemistry

We will use well-known elemental properties along with the criterion that compounds must not have an overall charge in order to sequentially apply different levels of screening and count the possible combinations:

1. Element combination counting - Considering combinations of elements and ignore oxidation states
2. Ion combination counting - Considering combinations of elements in their allowed oxidation states
3. Charge neutrality - Discarding any combinations that would not make a charge neutral compound
4. Electronegativity difference - Applying a 'common sense' filter based on element electronegativities

Most of the code is in the notebook explicitly; however, we are also importing an external set of python modules that has been designed exactly for this sort of task.

### Setting up and choosing the search-space

The code below just imports the elemental data that we need in order to do our counting. The main variable in the cell below for this practical is the ```max_atomic_number``` which dictates how many elements to consider. 

For example, when ```max_atomic_number = 10``` the elements from H to Ne are considered in the search. Whenever you change it, remember to run the cell again.

- ** Have a read through and run the code below (comments explaining the code are surrounded by #) **

In [ ]:
# Imports the SMACT toolkit for later on #
import smact.core as core

# Gets element data from file and puts into a list #
with open('Counting/element.txt','r') as f:
    data = f.readlines()

list_of_elements = []

# Specify the range of elements to include #

### EDIT BELOW ###
max_atomic_number = 10
##################

# Populates a list with the elements we are concerned with #
for line in data:
    if not line.startswith('#'):
        species = line.split()
        if int(species[0]) > 0 and int(species[0]) < max_atomic_number + 1:
            list_of_elements.append(species[1])
            
print '--- Considering the %s elements from %s to %s ---' %(len(list_of_elements), list_of_elements[0], list_of_elements[-1])

### 1. Element combination counting

This first procedure simply counts how many binary combinations are possible for a given set of elements. This is basically just a numerical problem, as we are not considering elemental properties in any way for the time being.

- **Read through and run the code below**
- **Increase the number of elements to consider (max_atomic_number in the cell above) to see how this affects the number of combinations**
- **If you can, extend the for loops below to add up ternary (and/or quaternary) combinations. It's advisable to change the number of elements to include back to 10 first!** 

**Hint: The next exercise is set up for ternary counting so you could come back and do this after completing that.**

In [ ]:
# Counts up possibilities and prints the output #
element_count = 0
for i, ele_a in enumerate(list_of_elements):
    element_count = element_count + 1
    for j, ele_b in enumerate(list_of_elements[i+1:]):
        element_count = element_count + 1
        print ele_a + ' ' + ele_b

print 'Number of combinations = ', element_count

### 2. Ion combination counting

As you can see the situation gets out of hand quite quickly. Even at a glance it should be obvious that many of the combinations are not feasable. This is largely to do with our expectation for elements to be in their common oxidation states. 

This procedure expands on the previous one by considering each known oxidation state of an element (so strictly speaking we are not dealing with 'ions').

It incorporates a library of known oxidation states for each element and is this time already set up to search for ternary combinations. The code prints out the combination of elements including their oxidation states. There is also a timer so that you can see how long it takes to run the program. 

- ** Reset the search space to ~10 elements and run the code below **
- ** Have a play with the code to change the search-space (max_atomic_number still in the top cell above).** 

**Hint: It is advisable to increase the search-space gradually and see how long the calculation takes each time. If you go straight for a big number you could be waiting a while for the calculation to run....**

In [ ]:
# Sets up the timer to see how long the program takes to run #
import time
start_time = time.time()

# Counts up the possibilities and prints the output #
ion_count = 0
for i, ele_a in enumerate(list_of_elements):
    for ox_a in core.Element(ele_a).oxidation_states:
        ion_count = ion_count + 1
        for j, ele_b in enumerate(list_of_elements[i+1:]):
            for ox_b in core.Element(ele_b).oxidation_states:
                ion_count = ion_count + 1
                for k, ele_c in enumerate(list_of_elements[i+2:]):
                    for ox_c in core.Element(ele_c).oxidation_states:
                        ion_count = ion_count + 1
                    
                        print '%s %s   %s %s   %s %s' %(ele_a, ox_a, ele_b, ox_b, ele_c, ox_c)

print 'Number of combinations = ', ion_count
print("--- %s seconds to run ---" % (time.time() - start_time))

All we seem to have done is make matters worse! 

We are effectively introducing many more species by further splitting each element in our search-space into separate species, one for each allowed oxidation state. When we get to max_atomic_number > 20, we are including the transition metals and their many oxidation states. 

Below are the consequences of running this for 100 elements for ternary combinations (on a pretty good workstation!):

<img src = 'Images/100atoms_ternary.png'>

### 3. Charge neutrality

However, the previous step is a necessary evil to incorporate our first filter which is that the compounds must be charge neutral overall. Scrolling through the output from above, it's easy to see that the vast majority of the combinations are not charge neutral overall. We can discard these combinations to start narrowing our search down to more 'sensible' (or at least not totally unreasonable) ones.

- ** Reset the search space to ~10 elements or so and run the code below **
- ** Increase the number of elements to consider again and compare the output of 1. and 2. with that of the below cell**

In [ ]:
# Sets up the timer to see how long the program takes to run #
import time
start_time = time.time()

# Counts up the possibilities #
charge_neutral_count = 0
for i, ele_a in enumerate(list_of_elements):
    for ox_a in core.Element(ele_a).oxidation_states:
        for j, ele_b in enumerate(list_of_elements[i+1:]):
            for ox_b in core.Element(ele_b).oxidation_states:
                for k, ele_c in enumerate(list_of_elements[i+2:]):
                    for ox_c in core.Element(ele_c).oxidation_states:
                        
# Checks if the combination is charge neutral before printing it out! #                        
                        cn_e, cn_r = core.charge_neutrality([ox_a, ox_b, ox_c], threshold=1)
                        if cn_e:
                            charge_neutral_count = charge_neutral_count + 1
                            print '%s %s   %s %s   %s %s' %(ele_a, ox_a, ele_b, ox_b, ele_c, ox_c)
print 'Number of combinations = ', charge_neutral_count
print("--- %s seconds to run ---" % (time.time() - start_time))                        

This drastically reduces the number of combinations we get out and we can even begin to see some compounds that we recognise and know exist. 

### 4. Electronegativity difference

The last step in this exercise is to incorporate the key chemical property of electronegativity i.e. the propensity of an element to attract electron density to itself in a bond. This is a logical step as inspection of the output from above reveals that some combinations feature a species in a higher (more positive) oxidation state which is more elecronegative than other species present, e.g.:

$B^{1+} O^{2+} N^{3-}$

*Electronegativities:*
B = 2.04
O = 3.44
N = 3.04

This is nonsensical because the concepts of electronegativity and oxidation state are linked to how electronic charge is arranged in a compound and they must not contradict each other. 

With this in mind, we now incorporate another filter which checks that the species with higher oxidation states have lower electronegativities. The library of values used is of the widely accepted electronegativity scale as developed by Linus Pauling. The scale is based on the dissociation energies of heteronuclear diatomic molecules and their corresponding homonuclear diatomic molecules:

<img src = 'Images/pauling-equation.png'>

- ** Run the code below. Again, feel free to adjust the search space.**

In [ ]:
# Sets up the timer to see how long the program takes to run #
import time
start_time = time.time()

# Counts up the possibilities and now also records the 'Pauling electronegativity' as it goes #
pauling_count = 0
for i, ele_a in enumerate(list_of_elements):
    paul_a = core.Element(ele_a).pauling_eneg
    for ox_a in core.Element(ele_a).oxidation_states:
        for j, ele_b in enumerate(list_of_elements[i+1:]):
            paul_b = core.Element(ele_b).pauling_eneg
            for ox_b in core.Element(ele_b).oxidation_states:
                for k, ele_c in enumerate(list_of_elements[i+2:]):
                    paul_c = core.Element(ele_c).pauling_eneg
                    for ox_c in core.Element(ele_c).oxidation_states:
                        oxidation_states = [ox_a, ox_b, ox_c]
                        pauling_electro = [paul_a, paul_b, paul_c]
                        electroneg_makes_sense = core.pauling_test(oxidation_states, pauling_electro)
# Checks if the combination is charge neutral before printing it out! #                        
                        cn_e, cn_r = core.charge_neutrality([ox_a, ox_b, ox_c], threshold=1)
                        if cn_e:
                            if electroneg_makes_sense:
# Checks if the combination meets the electronegativity criteria #
                                pauling_count = pauling_count + 1
                                print '%s %s   %s %s   %s %s' %(ele_a, ox_a, ele_b, ox_b, ele_c, ox_c)
                                
print 'Number of combinations =', pauling_count
print("--- %s seconds to run ---" % (time.time() - start_time))

Again, we have reduced the number of combinations in our output. 

- **The electronegativity check in the SMACT python module incorporates a tolerance factor which can also be tweaked. Why might this be necessary?**

### Speedy calculations 

For a given search-space of elements, the number of combinations in the output has decreased each time we've applied a filter. However, the time taken to carry out the calculation has increased. This highlights a fundamental trade-off, however there are some clever coding techniqes that can be used to considerably speed things up. 

Multi-threading, for example, reduced the time taken to carry out the same ternary 'Ion count' for 100 atoms as before from ~ 1 hour to just 8 seconds (carried out on the same workstation)!

<img src = 'Images/100atoms_threaded_ternary.png'>

Furthermore, a quaternary 'Ion count' for 100 atoms took only 15 minutes.

<img src = 'Images/100atoms_threaded_quaternary.png'>

## Some technicalities...

**A note on multiplicity:** In this exercise, we have not considered the multiplicity in our permutations at all, i.e. we have considered only AB, ABC (and ABCD for quaternary) type combinations. When extended to AxByCz... where x,y,z > 1, the numbers involved get considerably larger still.

**Permutation vs. Combination:** Depending on how you want to approach the combinatorial probelm, you need to decide to what extent two results are equivalent. For example, consider the results:

$ H^{-1} Li^{+2} B^{+1} $  
and  
$ H^{-1} B^{+1} Li^{+2} $ 
 
When cycling through **permutations** for a given set of elements they would both appear and count for one each because the B and Li species appear in a different order. However, when cycling through **combinations** they would not both appear, as they would be considered equivalent. Whichever is most useful depends on the task at hand and ties in to the idea of 'sites' in a compound (e.g. different A and B sites in a perovskite).

N.B. This is why the multi-threading ternary count above doesn't match exactly the number quoted for the same search in part 2.!  

** And some wisdom from our good friend, Linus: **

<img src = 'Images/linus-pauling.png'>